<a href="https://colab.research.google.com/github/gabrielvcruz/projeto_reducao_de_dimensionalidade/blob/main/projeto_Assistente_virtual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install pypiwin32
!pip install SpeechRecognition
!pip install gTTS
!pip install playsound
!pip install pyjokes
!pip install wikipedia
!apt-get update
!apt-get install -y portaudio19-dev
!pip install pyaudio
!pip install winshell
!pip install pygame
!pip install geocoder

  Using cached pypiwin32-223-py3-none-any.whl.metadata (236 bytes)
INFO: pip is looking at multiple versions of pypiwin32 to determine which version is compatible with other requirements. This could take a while.
  Using cached pypiwin32-219.zip (4.8 MB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backp

In [ ]:
import speech_recognition as sr
import os
import webbrowser
import wikipedia
import gTTS
import playsound
import requests
import geocoder
from datetime import datetime
from pygame import mixer

# Função de transformação de texto em fala (Text-to-Speech)
def speak(text, lang='pt'):
    tts = gTTS(text=text, lang=lang, slow=False)
    filename = "/content/voice.mp3"
    tts.save(filename)
    playsound.playsound(filename)

# Função de transformação de fala em texto (Speech-to-Text)
def get_audio():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Ajustando para ruído ambiente...")
        r.adjust_for_ambient_noise(source, duration=1)
        print("Por favor, fale...")
        audio = r.listen(source)
        said = ""
        try:
            said = r.recognize_google(audio, language='pt-BR')
            print(f"Você disse: {said}")
        except sr.UnknownValueError:
            speak("Desculpe, não consegui entender.")
        except sr.RequestError:
            speak("Desculpe, o serviço de reconhecimento de fala não está disponível.")
    return said.lower()

# Função para abrir o YouTube e buscar algo
def search_youtube(query):
    url = f"https://www.youtube.com/results?search_query={query}"
    webbrowser.open(url)
    speak(f"Aqui estão os resultados para {query} no YouTube.")

# Função para pesquisar na Wikipedia
def search_wikipedia(query):
    try:
        result = wikipedia.summary(query, sentences=3, lang='pt')
        speak("De acordo com a Wikipedia...")
        print(result)
        speak(result)
    except wikipedia.exceptions.DisambiguationError as e:
        speak(f"Existem várias opções para {query}, por favor seja mais específico.")
    except wikipedia.exceptions.HTTPTimeoutError:
        speak("Desculpe, houve um problema ao acessar a Wikipedia.")

# Função para mostrar a hora atual
def show_time():
    strTime = datetime.today().strftime("%H:%M %p")
    speak(f"A hora atual é {strTime}")

# Função para encontrar a farmácia mais próxima
def find_pharmacy():
    g = geocoder.ip('me')
    location = g.latlng
    if location:
        lat, lon = location
        url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat},{lon}&radius=5000&type=pharmacy&key=YOUR_GOOGLE_API_KEY"
        response = requests.get(url)
        pharmacies = response.json()
        if pharmacies['results']:
            pharmacy = pharmacies['results'][0]
            name = pharmacy['name']
            address = pharmacy['vicinity']
            speak(f"A farmácia mais próxima é {name}, localizada em {address}.")
        else:
            speak("Desculpe, não encontrei farmácias próximas.")
    else:
        speak("Não consegui determinar a sua localização.")

# Função principal para ouvir e responder aos comandos
def assistente_virtual():
    speak("Olá, como posso ajudar?")
    while True:
        text = get_audio()

        if 'youtube' in text:
            speak("O que você quer pesquisar no YouTube?")
            query = get_audio()
            search_youtube(query)

        elif 'pesquisar' in text:
            speak("O que você quer pesquisar na Wikipedia?")
            query = get_audio()
            search_wikipedia(query)

        elif 'hora' in text:
            show_time()

        elif 'farmácia' in text:
            find_pharmacy()

        elif 'sair' in text:
            speak("Até mais!")
            break
        else:
            speak("Desculpe, não entendi o que voçÊ falou.")

# Iniciar a assistente virtual
assistente_virtual()
